In [32]:
# Install packages 
import requests
import os
import pandas as pd
from dotenv import load_dotenv
import time
from transformers import pipeline
from langdetect import detect, DetectorFactory

In [2]:
# Load API keys from .env
load_dotenv()
NEWSAPI_KEY = os.getenv("NEWSAPI_KEY")

In [3]:
DetectorFactory.seed = 0  

In [4]:
# Define the NewsAPI endpoint for Forex news
url = f"https://newsapi.org/v2/everything?q=forex OR currency OR USD OR EUR OR forex trading&apiKey={NEWSAPI_KEY}"

# Send the request
response = requests.get(url)

# Convert response to JSON
data = response.json()
# Extract relevant fields from the API response
if response.status_code == 200:
    articles = data.get("articles", [])
    news_data = []
    for article in articles:
        try:
            language = detect(article["title"])
            if language == "en": 
                news_data.append({
                "news": article["title"]
                })
        except:
            pass
    

In [10]:
# Load FinBERT model
finbert = pipeline("sentiment-analysis", model="ProsusAI/finbert")
# Function to classify sentiment using FinBERT
def classify_sentiment(text):
    result = finbert(text)[0] 
    return result["label"]  

C:\Users\Admin\AppData\Roaming\Python\Python311\site-packages\huggingface_hub\file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [45]:
df = pd.read_csv("data/forex_news.csv")
df = pd.concat([df, pd.DataFrame({'news': articles, 'labels': list(map(classify_sentiment, news_data))})], axis=0, ignore_index=True)
df = df.drop_duplicates().reset_index(drop=True)  

In [46]:
df['news'] = df['news'].fillna('')
df=df[['news', 'labels']]
df.to_csv("data/forex_news.csv")